In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import spacy
from tqdm import tqdm
import json

/home/rootdaenib/anaconda3/envs/semanticSearch/lib/python3.10/site-packages/cupy/_environment.py:438: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda116

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
2022-10-15 18:30:17.551004: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow

In [2]:
driver = webdriver.Chrome(executable_path="./chromedriver")

In [3]:
driver.get("https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/index.html")


In [4]:
links = [href.find_element(By.TAG_NAME, 'a').get_attribute("href") for href in driver.find_element(By.ID, "toc_content").find_elements(By.TAG_NAME, "ul")]


In [5]:
links = [link for link in links if link is not None]

In [6]:
links = links[3:]

In [7]:
dataJson = {}

In [8]:
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_trf")

In [9]:
links

['https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/co-broker_deprecation.html#',
 'https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/co-about_logging_into_and_out_of_my_webmethods.html#',
 'https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/co-my_webmethods_user_interface.html#',
 'https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/ta-resizing_the_content_area_and_navigation_pane.html#',
 'https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/ta-closing_tabs.html#',
 'https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/re-monitoring_section_of_the_navigate_tab.html#',
 'https://documentation.softwa

In [10]:
dataJson = {}
exceptionLinks = list()
tqdmLinks = tqdm(links)
for link in tqdmLinks:
    try:
        driver.get(link)
        driver.switch_to.frame(0)
        title = driver.find_element(By.ID, "wwconnect_header").text
        if title not in dataJson:
            dataJson[title] = {}
        driver.execute_script("""
            var l = document.getElementById("wwconnect_header");
            l.parentNode.removeChild(l);
            """)
        divs = driver.find_elements(By.TAG_NAME, "div")
        dataJson[title]['data'] = []
        for div in divs[:-1]:
            if div.text != '':
                doc = nlp(div.text)
                for sent in doc.sents:
                    dataJson[title]['data'].append(sent.text)
        dataJson[title]['href'] = link
    except Exception as e:
        print(e)
        print(link)
        exceptionLinks.append(link)
        # break

 33%|███▎      | 68/203 [01:18<01:44,  1.29it/s]

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wwconnect_header"]"}
  (Session info: chrome=106.0.5249.103)

https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/re-configuration_files_stored_in_the_database.html#


 88%|████████▊ | 178/203 [03:29<00:46,  1.85s/it]

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wwconnect_header"]"}
  (Session info: chrome=106.0.5249.103)

https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/re-creating_a_my_webmethods_server_instance.html#wwID0EEHFV


100%|██████████| 203/203 [04:08<00:00,  1.23s/it]


In [11]:

with open("data.json", "a") as f:
    json.dump(dataJson, f)
with open("exceptionLinks.json", "a") as f:
    json.dump(exceptionLinks, f)

In [12]:
len(dataJson)

200

In [20]:
exceptionLinks

[]

In [21]:
len(links)

208

In [33]:
driver.switch_to.parent_frame()
driver.find_element(By.TAG_NAME, 'title').text

''

In [38]:
print(driver.execute_script("return document.title"))

API Gateway Cluster Configuration


In [2]:
dataJson = json.load(open("data.json", "r"))

In [3]:
for key in dataJson:
    dataJson[key]['data'].append(key)

In [5]:
json.dump(dataJson, open("data.json", "w"))